### Updating The FX_derivatives_all database 
Update with the TJ06 and TX06sap reports.

In [2]:
#Imports the daily TJ06 (options) & TX06 (forwards) and updates FX_Derivatives_All Database 
#which is located in the Financial_Database directory located within the Treasury drive

import pandas as pd
import csv
import datetime

#this function module returns the delta days for file date (ie. If today is Monday, we need Friday's date
#for the SAP file)
def whatday():
    wkday = datetime.datetime.today().weekday()
    if wkday == 0:    
        return 3
    else:
        return 1

#Putting into a variable date to be used with the file names
today = datetime.date.today()
prevday = today - datetime.timedelta(days=whatday())
prevday = prevday.strftime('%m%d%Y')

#Build file paths to get the daily TJ06 (options) and TX06 (forwards) files. 
dir_path = r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256'
file1_name = '\TX06_Extract' + prevday + '.txt'
file2_name = '\TJ06_Extract' + prevday + '.txt'
file_path1 = dir_path + file1_name
file_path2 = dir_path + file2_name

#Formating the header for daily TJ06 (options) and TX06 (forwards) files.
df1 = pd.read_csv(file_path1, delimiter = '|', skiprows=[0,2], header =[0])
df2 = pd.read_csv(file_path2, delimiter = '|', header =[0])

#Aligned header to match data columns for TJ06 (Option) report and removed blank spaces on header fields.
names = list(df2.columns.values)
newname = []
for name in names:
    newname.append(name.strip())
newname.append(newname.pop(0))
df2.columns = newname
df2.reset_index()


df2.insert(0,"ID Entity", 'ADIR')
df2.dropna(subset = ["Transactn"], inplace=True)

df2.reset_index(drop=True, inplace=True)

df2.dropna(axis=1, how='all', inplace=True)

#create dictionary of counter party names and their associated numbers
counter_party_dicts = {'61':'JPM',
                       '2':'BOA',
                       '2081':'USB',
                       '2121':'WFB',
                       '2101':'HSBC',
                       '12600': 'SG',
                       '1200000141': 'BOA',
                       '2131':'MSC',
                       '3':'UBS',
                       '5':'GS',
                       '72':'CITI',
                       '10020':'RBS'}


    
#df = pd.read_csv(r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256\TX06_Extract09212020.txt',

In [ ]:
#trying to convert to string values
cps = set(df2['Counterparty'].str().values)
#cps = np.array2string(cps, precision = 0, separator = ',')

